### Import bibliotek

In [22]:
import os
import json
import requests
from bs4 import BeautifulSoup


### Narzędzia

In [23]:
def extract(ancestor, selector, attribute=None, many=False):
    if selector:
        if many:
            if attribute:
                return [item[attribute].strip() for item in ancestor.select(selector)]
            return [item.text.strip() for item in ancestor.select(selector)]
        if attribute:
            try:
                return  ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:   
            return  ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        try:
            return ancestor[attribute]
        except TypeError:
            return None
    return ancestor.text.strip()

### Pobieranie ze strony ceneo.pl opinii o konkretnym produkcie

In [24]:
with open("./headers.json", "r") as jf:
    headers = json.load(jf)


In [27]:
review_scheme = {
    "review_id": (None, "data-entry-id"),
    "author": ("span.user-post__author-name",),
    "recomendation": ("span.user-post__author-recomendation > em",),
    "stars": ("span.user-post__score-count",),
    "content": ("div.user-post__text",),
    "pros": ("div.review-feature__item--positive", None, True),
    "cons": ("div.review-feature__item--negative", None, True),
    "likes": ("button.vote-yes > span",),
    "dislikes": ("button.vote-no > span",),
    "publish_date": ("span.user-post__published > time:nth-child(1)", 'datetime'),
    "purchase_date": ("span.user-post__published > time:nth-child(2)", 'datetime'),
    }

In [32]:
product_id = "17642771"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_reviews = []
while next_page:
    response = requests.get(next_page, headers=headers)
    print(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, 'html.parser')
        reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(reviews))
        for review in reviews:
            single_review = {
                key: extract(review, *value)
                for key, value in review_scheme.items()
            }
            all_reviews.append(single_review)
        try:
            next_page = "https://www.ceneo.pl"+extract(page_dom, "a.pagination__next", "href")
        except TypeError:
            next_page = None
print(all_reviews)
        

https://www.ceneo.pl/17642771#tab=reviews
2
[{'review_id': '2606973', 'author': 'Iwona', 'recomendation': 'Polecam', 'stars': '5/5', 'content': 'Super smak majonezu, nie znam lepszego, no może oprócz tego, który zrobię sama. Smakowity, bez konserwantów i sztucznych dodatków. Słoiczek w sam raz na sałatkę dla całej rodziny. Bardzo szybko znika z mojej lodówki. Polecam. Duży plus za polską jakość.', 'pros': [], 'cons': [], 'likes': '2', 'dislikes': '0', 'publish_date': '2014-06-16 17:27:41', 'purchase_date': '2014-06-03 20:34:38'}, {'review_id': '2511672', 'author': 'Użytkownik Ceneo', 'recomendation': None, 'stars': '3/5', 'content': 'Kupiłam tylko raz i to był jedyny raz. intensywny smak, totalnie nie w moim guście. Moim zdaniem dekoracyjny o wiele lepszy', 'pros': [], 'cons': [], 'likes': '0', 'dislikes': '2', 'publish_date': '2014-04-01 17:49:17', 'purchase_date': None}]


### Zapisanie wszystkich opinii o konkretnym produkcie w bazie danych

In [29]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [30]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_reviews, jf, indent=4, ensure_ascii=False)